In [1]:
import findspark
findspark.init('C:\\spark')

## insanlar.json

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("First_Example").getOrCreate()
df = spark.read.json(".\\Pyspark_Humans_Data\\insanlar.json")
df.show(50)

+--------------------+--------+----+--------+----+
|     _corrupt_record|cinsiyet|  id|    isim| yas|
+--------------------+--------+----+--------+----+
|[{"id":1,"isim":"...|    null|null|    null|null|
|                null|  Female|   2|  Nadean|  29|
|                null|    Male|   3| Edgardo|  67|
|                null|  Female|   4|   Vonny|  59|
|                null|    Male|   5|Hercules|  62|
|                null|    Male|   6|   Lanny|  65|
|                null|    Male|   7|  Norris|  19|
|                null|  Female|   8|   Karia|  51|
|                null|  Female|   9|  Korrie|  37|
|                null|  Female|  10|  Hallie|  54|
|                null|  Female|  11|   Aidan|  31|
|                null|  Female|  12|   Lotte|  60|
|                null|  Female|  13|    Zora|  28|
|                null|  Female|  14|Odelinda|  46|
|                null|    Male|  15|Basilius|  51|
|                null|  Female|  16|   Irena|  52|
|                null|    Male|

In [4]:
df.select('yas').show(5)

+----+
| yas|
+----+
|null|
|  29|
|  67|
|  59|
|  62|
+----+
only showing top 5 rows



In [5]:
df.select(df['isim'],df['yas']+10).show(5)
# df.select(df.isim, df.yas+10).show(5)

+--------+----------+
|    isim|(yas + 10)|
+--------+----------+
|    null|      null|
|  Nadean|        39|
| Edgardo|        77|
|   Vonny|        69|
|Hercules|        72|
+--------+----------+
only showing top 5 rows



In [6]:
df.filter(df.yas<22).orderBy('yas', ascending=False).show(10)

+---------------+--------+---+--------+---+
|_corrupt_record|cinsiyet| id|    isim|yas|
+---------------+--------+---+--------+---+
|           null|  Female| 58|Tatiania| 21|
|           null|  Female|413|   Dawna| 21|
|           null|  Female| 83|  Marthe| 21|
|           null|    Male|185|   Quint| 21|
|           null|  Female| 49|  Florri| 21|
|           null|  Female|312| Gunilla| 21|
|           null|  Female|233|  Britte| 20|
|           null|  Female|101| Bernita| 20|
|           null|  Female|281|  Nancey| 20|
|           null|    Male|340|   Micah| 20|
+---------------+--------+---+--------+---+
only showing top 10 rows



In [7]:
df.groupBy('yas').count().orderBy('count', ascending=False).show(10)

+---+-----+
|yas|count|
+---+-----+
| 29|   15|
| 45|   15|
| 16|   13|
| 43|   13|
| 39|   13|
| 51|   12|
| 15|   12|
| 50|   12|
|  9|   12|
| 41|   11|
+---+-----+
only showing top 10 rows



## ratings.csv

https://www.kaggle.com/datasets/zygmunt/goodbooks-10k

In [17]:
import findspark
findspark.init('C:\\spark')

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName("SQL_BOOKS").getOrCreate()

In [19]:
def CSV_ROW(row):
    item = row.split(',')
    return Row(book_id=int(item[0]), user_id=int(item[1]), rating=int(item[2]))

In [20]:
rows = spark.sparkContext.textFile(".\\Pyspark_Books_Data\\ratings.csv")
header = rows.first()
header

'book_id,user_id,rating'

In [21]:
rows = rows.filter(lambda x: x!= header)
books = rows.map(CSV_ROW)

In [23]:
df = spark.createDataFrame(books).cache()
df.createOrReplaceTempView("books")
df.show(10)

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
|      1|   2077|     4|
|      1|   2487|     4|
|      1|   2900|     5|
|      1|   3662|     4|
|      1|   3922|     5|
+-------+-------+------+
only showing top 10 rows



In [25]:
# Datasette kac satir var?
df.count()

981756

In [26]:
# Datasette uniq kac satir var?
df.distinct().count()

980112

In [27]:
# Datasette uniq kac kitap var?
df.groupBy('book_id').count().distinct().count()

10000

In [28]:
# Alternative
from pyspark.sql.functions import countDistinct
df.select(countDistinct("book_id")).show()

+-----------------------+
|count(DISTINCT book_id)|
+-----------------------+
|                  10000|
+-----------------------+



In [29]:
# Her kitabin kac defa oylandigini bulalim.
df.groupBy('book_id').count().orderBy('count', ascending=False).show(10)

+-------+-----+
|book_id|count|
+-------+-----+
|     65|  100|
|    191|  100|
|    418|  100|
|    541|  100|
|    558|  100|
|   1010|  100|
|   1224|  100|
|   1258|  100|
|   1277|  100|
|   1360|  100|
+-------+-----+
only showing top 10 rows



In [30]:
# 100 den az veya fazla oylanmis olan kitap sayisi nedir?
df2 = df.groupBy('book_id').count().orderBy('count', ascending=False)
df2.filter(df2['count']!=100).count()

2544

In [31]:
# uniq puanlama sayilarinin frekansa gore siralamasi
df2.groupBy('count').count().orderBy('count', ascending=False).show(60)

+-----+-----+
|count|count|
+-----+-----+
|  100| 7456|
|   99|  744|
|   98|  329|
|   97|  245|
|   96|  159|
|   95|  148|
|   94|   93|
|   93|   90|
|   92|   74|
|   91|   58|
|   90|   55|
|   89|   58|
|   88|   39|
|   87|   38|
|   86|   37|
|   85|   35|
|   84|   31|
|   83|   28|
|   82|   17|
|   81|   23|
|   80|   12|
|   79|   15|
|   78|   19|
|   77|   14|
|   76|   21|
|   75|   11|
|   74|    8|
|   73|   12|
|   72|    7|
|   71|    8|
|   70|   13|
|   69|   12|
|   68|    4|
|   67|   10|
|   66|   11|
|   65|   10|
|   64|    4|
|   63|    6|
|   62|    3|
|   61|    7|
|   60|    5|
|   59|    5|
|   58|    1|
|   57|    4|
|   56|    1|
|   55|    2|
|   54|    3|
|   53|    2|
|   52|    3|
|   48|    1|
|   47|    1|
|   46|    1|
|   44|    1|
|   41|    1|
|   36|    1|
|   34|    1|
|   24|    1|
|   11|    1|
|    8|    1|
+-----+-----+



In [32]:
# rating i 4 ve uzeri kitaplari getir
spark.sql("SELECT * FROM books WHERE rating>=4").groupBy('book_id').count().orderBy('count', ascending=False).show(10)

+-------+-----+
|book_id|count|
+-------+-----+
|   2244|   98|
|   3628|   98|
|   6642|   97|
|   1308|   97|
|   9566|   97|
|   4483|   97|
|   1904|   97|
|   6920|   96|
|   6527|   96|
|   3275|   96|
+-------+-----+
only showing top 10 rows



In [33]:
# Alternative
df.filter(df['rating']>=4).groupBy('book_id').count().orderBy('count', ascending=False).show(10)

+-------+-----+
|book_id|count|
+-------+-----+
|   2244|   98|
|   3628|   98|
|   6642|   97|
|   1904|   97|
|   1308|   97|
|   9566|   97|
|   4483|   97|
|   3275|   96|
|   6920|   96|
|   3660|   96|
+-------+-----+
only showing top 10 rows



In [34]:
# rating i 2 nin alti kitaplari getir
spark.sql("SELECT * FROM books WHERE rating < 2").groupBy('book_id').count().orderBy('count', ascending=False).show(10)

+-------+-----+
|book_id|count|
+-------+-----+
|   1793|   39|
|     34|   29|
|   1409|   26|
|      3|   23|
|   4009|   23|
|   1822|   22|
|   3550|   22|
|   1338|   22|
|   8466|   21|
|   4399|   21|
+-------+-----+
only showing top 10 rows



In [35]:
# Alternative
df.filter(df['rating']<2).groupBy('book_id').count().orderBy('count', ascending=False).show(10)

+-------+-----+
|book_id|count|
+-------+-----+
|   1793|   39|
|     34|   29|
|   1409|   26|
|      3|   23|
|   4009|   23|
|   1822|   22|
|   3550|   22|
|   1338|   22|
|   4399|   21|
|   8466|   21|
+-------+-----+
only showing top 10 rows



In [36]:
# Ortalama ratingi en yuksek 10 kitabi getir
df.groupBy('book_id').mean('rating').orderBy('avg(rating)', ascending=False).show(10)

+-------+-----------------+
|book_id|      avg(rating)|
+-------+-----------------+
|   7947|4.820224719101123|
|   5207|             4.78|
|   6920|             4.78|
|   9566|4.777777777777778|
|   8946|4.774193548387097|
|   6361|             4.77|
|   3275|             4.77|
|   6590|             4.75|
|   4483|             4.75|
|   5580|             4.75|
+-------+-----------------+
only showing top 10 rows



In [37]:
# Ortalama ratingi en dusuk 10 kitabi getir
df.groupBy('book_id').mean('rating').orderBy('avg(rating)', ascending=True).show(10)

+-------+------------------+
|book_id|       avg(rating)|
+-------+------------------+
|   1793|              1.96|
|   4045| 2.235294117647059|
|   7636|            2.3125|
|   1822| 2.350515463917526|
|   1409|              2.43|
|   4399|2.4574468085106385|
|   4991| 2.462686567164179|
|   3550|              2.49|
|   4009|              2.53|
|   4283|              2.54|
+-------+------------------+
only showing top 10 rows



In [38]:
# Alternative
spark.sql("SELECT * FROM books").groupBy('book_id').mean('rating').orderBy('avg(rating)', ascending=True).show(10)

+-------+------------------+
|book_id|       avg(rating)|
+-------+------------------+
|   1793|              1.96|
|   4045| 2.235294117647059|
|   7636|            2.3125|
|   1822| 2.350515463917526|
|   1409|              2.43|
|   4399|2.4574468085106385|
|   4991| 2.462686567164179|
|   3550|              2.49|
|   4009|              2.53|
|   4283|              2.54|
+-------+------------------+
only showing top 10 rows



In [39]:
df.show()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
|      1|   2077|     4|
|      1|   2487|     4|
|      1|   2900|     5|
|      1|   3662|     4|
|      1|   3922|     5|
|      1|   5379|     5|
|      1|   5461|     3|
|      1|   5885|     5|
|      1|   6630|     5|
|      1|   7563|     3|
|      1|   9246|     1|
|      1|  10140|     4|
|      1|  10146|     5|
|      1|  10246|     4|
|      1|  10335|     4|
+-------+-------+------+
only showing top 20 rows



## Video_Games_Sales_as_at_22_Dec_2016.csv

https://www.kaggle.com/datasets/rush4ratio/video-game-sales-with-ratings

In [45]:
import findspark
findspark.init('C:\\spark')

In [46]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [54]:
dataset = spark.read.format("csv").option("header", "true").load(".\\Pyspark_Video_Games_Data\\Video_Games_Sales_as_at_22_Dec_2016.csv")
dataset

DataFrame[Name: string, Platform: string, Year_of_Release: string, Genre: string, Publisher: string, NA_Sales: string, EU_Sales: string, JP_Sales: string, Other_Sales: string, Global_Sales: string, Critic_Score: string, Critic_Count: string, User_Score: string, User_Count: string, Developer: string, Rating: string]

In [55]:
data = dataset.groupby("Platform").count().orderBy("count", ascending = False)
data.collect()

[Row(Platform='PS2', count=2161),
 Row(Platform='DS', count=2152),
 Row(Platform='PS3', count=1331),
 Row(Platform='Wii', count=1320),
 Row(Platform='X360', count=1262),
 Row(Platform='PSP', count=1209),
 Row(Platform='PS', count=1197),
 Row(Platform='PC', count=974),
 Row(Platform='XB', count=824),
 Row(Platform='GBA', count=822),
 Row(Platform='GC', count=556),
 Row(Platform='3DS', count=520),
 Row(Platform='PSV', count=432),
 Row(Platform='PS4', count=393),
 Row(Platform='N64', count=319),
 Row(Platform='XOne', count=247),
 Row(Platform='SNES', count=239),
 Row(Platform='SAT', count=173),
 Row(Platform='WiiU', count=147),
 Row(Platform='2600', count=133),
 Row(Platform='NES', count=98),
 Row(Platform='GB', count=98),
 Row(Platform='DC', count=52),
 Row(Platform='GEN', count=29),
 Row(Platform='NG', count=12),
 Row(Platform='SCD', count=6),
 Row(Platform='WS', count=6),
 Row(Platform='3DO', count=3),
 Row(Platform='TG16', count=2),
 Row(Platform='PCFX', count=1),
 Row(Platform='GG', 

In [56]:
data = dataset.groupBy('Genre').count().orderBy('count',ascending=False)
data.collect()

[Row(Genre='Action', count=3370),
 Row(Genre='Sports', count=2348),
 Row(Genre='Misc', count=1750),
 Row(Genre='Role-Playing', count=1500),
 Row(Genre='Shooter', count=1323),
 Row(Genre='Adventure', count=1303),
 Row(Genre='Racing', count=1249),
 Row(Genre='Platform', count=888),
 Row(Genre='Simulation', count=874),
 Row(Genre='Fighting', count=849),
 Row(Genre='Strategy', count=683),
 Row(Genre='Puzzle', count=580),
 Row(Genre=None, count=2)]

In [57]:
rows = spark.sparkContext.textFile(".\\Pyspark_Video_Games_Data\\Video_Games_Sales_as_at_22_Dec_2016.csv")
header = rows.first()
header.split(',')

['Name',
 'Platform',
 'Year_of_Release',
 'Genre',
 'Publisher',
 'NA_Sales',
 'EU_Sales',
 'JP_Sales',
 'Other_Sales',
 'Global_Sales',
 'Critic_Score',
 'Critic_Count',
 'User_Score',
 'User_Count',
 'Developer',
 'Rating']

In [58]:
def CSV_ROW(row):
    global header
    a=header.split(',')
    item = row.split(',')
    return Row(Name=item[0], Platform=item[1], Year_of_Release=item[2], Genre=item[3], 
               Publisher=item[4], EU_Sales=item[6], Developer=item[13], Rating=item[14])

In [59]:
rows = rows.filter(lambda x: x!= header)
books = rows.map(CSV_ROW)

In [60]:
df = spark.createDataFrame(books).cache()
df.createOrReplaceTempView('books')
df.show(10)

+--------------------+--------+---------------+------------+---------+--------+---------+--------+
|                Name|Platform|Year_of_Release|       Genre|Publisher|EU_Sales|Developer|  Rating|
+--------------------+--------+---------------+------------+---------+--------+---------+--------+
|          Wii Sports|     Wii|           2006|      Sports| Nintendo|   28.96|      322|Nintendo|
|   Super Mario Bros.|     NES|           1985|    Platform| Nintendo|    3.58|         |        |
|      Mario Kart Wii|     Wii|           2008|      Racing| Nintendo|   12.76|      709|Nintendo|
|   Wii Sports Resort|     Wii|           2009|      Sports| Nintendo|   10.93|      192|Nintendo|
|Pokemon Red/Pokem...|      GB|           1996|Role-Playing| Nintendo|    8.89|         |        |
|              Tetris|      GB|           1989|      Puzzle| Nintendo|    2.26|         |        |
|New Super Mario B...|      DS|           2006|    Platform| Nintendo|    9.14|      431|Nintendo|
|         